In [ ]:
import torch # Si importo sólo los módulos, me sale error
import os # Si importo sólo los módulos, me sale error
import cv2 # Si importo sólo los módulos, me sale error
import numpy as np # Si importo sólo los módulos, me sale error
from numpy import array, ndarray, squeeze
from PIL import Image, ImageOps
from pandas import DataFrame, read_csv
from os import listdir, path
from os.path import isdir, join, isfile, dirname
from cv2 import imread, imwrite, resize, cvtColor, COLOR_BGR2GRAY
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms
from collections import OrderedDict
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
# Defino las funciones individuales

def construir_ruta_imagen(nombre_imagen, carpeta_principal_imagenes, subcarpeta=None, categoria=None):
    if subcarpeta and categoria:
        return os.path.join(carpeta_principal_imagenes, subcarpeta, categoria, nombre_imagen)
    elif subcarpeta:
        return os.path.join(carpeta_principal_imagenes, subcarpeta, nombre_imagen)
    else:
        return os.path.join(carpeta_principal_imagenes, nombre_imagen)

def nuevo_alto_deseado(imagen):
    alto_original, ancho_original, _ = imagen.shape
    if alto_original<670:
      nuevo_alto_deseado = 447
    elif alto_original>1200:
      nuevo_alto_deseado = 1440
    else:
      nuevo_alto_deseado = 800
    return nuevo_alto_deseado

def redimensionar_imagenes(imagen, nuevo_alto_deseado):
    alto_original, ancho_original, _ = imagen.shape
    nuevo_alto_deseado =  nuevo_alto_deseado(imagen)
    factor_redimensionamiento = nuevo_alto_deseado / alto_original
    nuevo_ancho = int(ancho_original * factor_redimensionamiento)
    imagen_redimensionada = cv2.resize(imagen, (nuevo_ancho, nuevo_alto_deseado))
    return imagen_redimensionada


def convertir_a_modo_L(imagen):
    if isinstance(imagen, np.ndarray):
        imagen_pil = Image.fromarray(imagen)
        imagen_gris = imagen_pil.convert("L")
        return np.array(imagen_gris)

def add_padding(image, target_size=1440):
    ancho, alto = image.size
    alto, ancho = torch.tensor(alto), torch.tensor(ancho)  # Convertir a tensores de PyTorch
    padding_vertical = max((target_size - alto) // 2, torch.tensor(0))
    padding_horizontal = max((target_size - ancho) // 2, torch.tensor(0))
    imagen_con_padding = ImageOps.expand(image, (padding_horizontal, padding_vertical, padding_horizontal, padding_vertical), fill=0)
    imagen_con_padding = imagen_con_padding.resize((target_size, target_size))
    return imagen_con_padding

def procesar_imagen_padding(image_paths, output_directory):
    nuevas_rutas = []
    max_size = 1440

    for image_path in image_paths:
        imagen = Image.open(image_path)
        imagen_con_padding = add_padding(imagen, max_size)
        imagen_con_padding.save(image_path)  # Sobrescribir la imagen original con la versión con padding
        nuevas_rutas.append(image_path)
    return nuevas_rutas

def redimensionar_imagenes_con_padding(image_paths, nuevo_alto=720, nuevo_ancho=720):

    nuevas_rutas2 = []

    for image_path in image_paths:
        imagen = cv2.imread(image_path)
        if imagen is not None:
            imagen_redimensionada = cv2.resize(imagen, (nuevo_ancho, nuevo_alto))
            cv2.imwrite(image_path, imagen_redimensionada)
            nuevas_rutas2.append(image_path)
    return nuevas_rutas2

def normalizar_imagenes(ruta):
    imagen = load_img(ruta)
    imagen = imagen.convert("L")
    imagen_array = img_to_array(imagen)
    imagen_normalizada = imagen_array / 255.0
    return imagen_normalizada

def procesar_imagen(nombre_imagen, carpeta_principal_imagenes):
    ruta_imagen = construir_ruta_imagen(nombre_imagen, carpeta_principal_imagenes)
    carpeta_imagen_original = os.path.dirname(ruta_imagen)
    imagen = cv2.imread(ruta_imagen)
    imagen_redimensionada = redimensionar_imagenes(imagen, nuevo_alto_deseado)
    imagen_gris = convertir_a_modo_L(imagen_redimensionada)
    ruta_imagen_gris=cv2.imwrite(ruta_imagen,imagen_gris)
    imagen_con_padding = procesar_imagen_padding([ruta_imagen], carpeta_imagen_original)
    imagen_redimensionada2 = redimensionar_imagenes_con_padding(imagen_con_padding, nuevo_alto=720, nuevo_ancho=720)
    imagen_normalizada = normalizar_imagenes(ruta_imagen)
    imagen_normalizada_squeezed = np.squeeze(imagen_normalizada, axis=2)
    imagen_tensor = torch.tensor(imagen_normalizada_squeezed, dtype=torch.float32)
    imagen_tensor = imagen_tensor.unsqueeze(0).unsqueeze(0)
    return imagen_tensor

def predecir_neumonia(modelo, imagen_tensor):
    modelo.eval()
    with torch.no_grad():
        salida = modelo(imagen_tensor)
        _, prediccion = torch.max(salida, 1)
    return prediccion.item()

In [ ]:
# Defino el modelo
class DenseLayer(nn.Sequential):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.add_module('bn1', nn.BatchNorm2d(in_channels))
        self.add_module('relu1', nn.ReLU(inplace=True))
        self.add_module('conv1', nn.Conv2d(in_channels, 4 * growth_rate, kernel_size=1, stride=1, bias=False))
        self.add_module('bn2', nn.BatchNorm2d(4 * growth_rate))
        self.add_module('relu2', nn.ReLU(inplace=True))
        self.add_module('conv2', nn.Conv2d(4 * growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False))

class DenseBlock(nn.Sequential):
    def __init__(self, num_layers, in_channels, growth_rate):
        super(DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = DenseLayer(in_channels + i * growth_rate, growth_rate)
            self.add_module('denselayer%d' % (i + 1), layer)

class TransitionLayer(nn.Sequential):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.add_module('bn', nn.BatchNorm2d(in_channels))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))

class DenseNet(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16), num_init_features=64, num_classes=2):
        super(DenseNet, self).__init__()

        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),
            ('bn0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(num_layers=num_layers, in_channels=num_features, growth_rate=growth_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = TransitionLayer(in_channels=num_features, out_channels=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        self.features.add_module('bn5', nn.BatchNorm2d(num_features))
        self.features.add_module('relu5', nn.ReLU(inplace=True))
        self.features.add_module('avgpool5', nn.AdaptiveAvgPool2d((1, 1)))

        self.classifier = nn.Linear(num_features, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.GroupNorm):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = torch.flatten(features, 1)
        out = self.classifier(out)
        return out

class CustomDenseNet(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomDenseNet, self).__init__()
        self.densenet = models.densenet121(pretrained=True)
        self.densenet.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.densenet(x)

In [ ]:
# Pruebo el modelo con imágenes nuevas

carpeta_principal_imagenes = "/content/drive/MyDrive/Proyecto modulo 7/prueba/imagenes_nuevas/"
nombre_imagenes = ["person1000_virus_1681.jpeg", "person1000_bacteria_2931.jpeg"]

if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    modelo = CustomDenseNet(num_classes=2)
    modelo.load_state_dict(torch.load('/content/drive/MyDrive/Proyecto modulo 7/prueba/mejor_modelo.pth'))
    modelo.to(device)

    resultados=[]

    for nombre_imagen in nombre_imagenes:
        imagen_tensor = procesar_imagen(nombre_imagen, carpeta_principal_imagenes)
        if imagen_tensor is not None:
            imagen_tensor = imagen_tensor.to(device)
            prediccion = predecir_neumonia(modelo, imagen_tensor)
            if prediccion == 1:
                resultados.append("La imagen {} muestra signos de neumonía.".format(nombre_imagen))
            else:
                resultados.append("La imagen {} no muestra signos de neumonía.".format(nombre_imagen))
        else:
            resultados.append("Error: No se pudo procesar la imagen {}.".format(nombre_imagen))

    for resultado in resultados:
        print(resultado)

La imagen person1000_virus_1681.jpeg muestra signos de neumonía.
La imagen person1000_bacteria_2931.jpeg muestra signos de neumonía.


# **API**

In [ ]:
%%writefile api.py
from flask import Flask, jsonify, request
import cv2
import torch
from modelo import CustomDenseNet, procesar_imagen, predecir_neumonia  # Ajusta la importación según tu estructura de proyecto

app = Flask(__name__)

# Cargar el modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelo = CustomDenseNet(num_classes=2)
modelo.load_state_dict(torch.load('/content/drive/MyDrive/Proyecto modulo 7/prueba/mejor_modelo.pth'))
modelo.to(device)

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file provided"}), 400

    file = request.files['file']

    temp_image_path = "temp_image.jpg"
    file.save(temp_image_path)
    imagen = cv2.imread(temp_image_path)

    imagen_tensor = procesar_imagen(imagen)
    if imagen_tensor is None:
        return jsonify({"error": "Error processing the image"}), 500

    imagen_tensor = imagen_tensor.to(device)

    prediccion = predecir_neumonia(modelo, imagen_tensor)

    if prediccion == 1:
        result = "La imagen muestra signos de neumonía."
    else:
        result = "La imagen no muestra signos de neumonía."

    return jsonify({"response": result})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)


Overwriting api.py
